<a href="https://colab.research.google.com/github/mandeep111/AI-Bank-Statement-Document-Automation-By-LLM-And-Personal-Finanical-Analysis-Prediction/blob/main/comfy_ui_live.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ------------------------------
# SYSTEM SETUP
# ------------------------------
!apt -y update
!apt -y install libgl1 wget unzip git

# ------------------------------
# INSTALL COMFYUI
# ------------------------------
!git clone https://github.com/comfyanonymous/ComfyUI
%cd /content/ComfyUI
!pip install -r requirements.txt --quiet

# ------------------------------
# FIX SOXR ERROR (Critical nanobind error)
# ------------------------------
# Uninstall conflicting packages and reinstall with correct versions
!pip uninstall -y soxr librosa soundfile
!pip install soxr==0.3.7 librosa==0.10.1 soundfile --quiet

# ------------------------------
# INSTALL CUSTOM NODES
# ------------------------------
%cd /content/ComfyUI/custom_nodes

# Essential nodes for your workflow
!git clone https://github.com/ltdrdata/ComfyUI-Manager
!git clone https://github.com/cubiq/ComfyUI_IPAdapter_plus    # IPAdapter (use plus version)
!git clone https://github.com/Fannovel16/comfyui_controlnet_aux  # ControlNet preprocessors
!git clone https://github.com/ssitu/ComfyUI_UltimateSDUpscale    # Ultimate SD Upscale

# Optional nodes (if you want face swap and inpainting features)
# !git clone https://github.com/Gourieff/ComfyUI-ReActor    # ReActorFaceSwap
# !git clone https://github.com/ltdrdata/ComfyUI-Impact-Pack # Additional nodes
# !git clone https://github.com/Uminosachi/ComfyUI-Inpaint-Anything

# ------------------------------
# INSTALL CUSTOM NODE DEPENDENCIES
# ------------------------------
print("Installing IPAdapter dependencies...")
%cd /content/ComfyUI/custom_nodes/ComfyUI_IPAdapter_plus
!pip install -r requirements.txt --quiet

print("Installing ControlNet Aux dependencies...")
%cd /content/ComfyUI/custom_nodes/comfyui_controlnet_aux
!pip install -r requirements.txt --quiet

print("Installing Ultimate SD Upscale dependencies...")
%cd /content/ComfyUI/custom_nodes/ComfyUI_UltimateSDUpscale
!pip install -r requirements.txt --quiet

# ------------------------------
# CREATE MODEL DIRECTORIES
# ------------------------------
!mkdir -p /content/ComfyUI/models/checkpoints
!mkdir -p /content/ComfyUI/models/loras
!mkdir -p /content/ComfyUI/models/controlnet
!mkdir -p /content/ComfyUI/models/ipadapter
!mkdir -p /content/ComfyUI/models/clip_vision
!mkdir -p /content/ComfyUI/models/upscale_models
!mkdir -p /content/ComfyUI/models/vae

# ------------------------------
# DOWNLOAD SD 1.5 MODELS (For your workflow)
# ------------------------------
print("Downloading models... This may take a while.")

# Checkpoint - EpicRealism (SD 1.5 with baked VAE)
print("📥 Downloading EpicRealism checkpoint...")
!wget -q --show-progress -O /content/ComfyUI/models/checkpoints/epicrealism_naturalSinRC1VAE.safetensors \
  "https://civitai.com/api/download/models/134065?type=Model&format=SafeTensor&size=pruned&fp=fp16"

# Alternative: Realistic Vision V5.1 (if above fails)
# !wget -q --show-progress -O /content/ComfyUI/models/checkpoints/realisticVisionV51.safetensors \
#   "https://civitai.com/api/download/models/130072"

# ControlNet - OpenPose
print("📥 Downloading OpenPose ControlNet...")
!wget -q --show-progress -O /content/ComfyUI/models/controlnet/control_v11p_sd15_openpose.pth \
  https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_openpose.pth

# IP-Adapter Plus for SD 1.5
print("📥 Downloading IP-Adapter Plus SD1.5...")
!wget -q --show-progress -O /content/ComfyUI/models/ipadapter/ip-adapter-plus_sd15.safetensors \
  https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-plus_sd15.safetensors

# IP-Adapter Plus Face for SD 1.5 (better for character consistency)
print("📥 Downloading IP-Adapter Plus Face SD1.5...")
!wget -q --show-progress -O /content/ComfyUI/models/ipadapter/ip-adapter-plus-face_sd15.safetensors \
  https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-plus-face_sd15.safetensors

# CLIP Vision Model (required for IP-Adapter)
print("📥 Downloading CLIP Vision model...")
!wget -q --show-progress -O /content/ComfyUI/models/clip_vision/CLIP-ViT-H-14-laion2B-s32B-b79K.safetensors \
  https://huggingface.co/h94/IP-Adapter/resolve/main/models/image_encoder/model.safetensors

# Upscaler - 4x UltraSharp
print("📥 Downloading 4x-UltraSharp upscaler...")
!wget -q --show-progress -O /content/ComfyUI/models/upscale_models/4x-UltraSharp.pth \
  https://huggingface.co/Kim2091/UltraSharp/resolve/main/4x-UltraSharp.pth

# LoRA - EpicRealism Helper (optional but recommended)
print("📥 Downloading EpicRealism Helper LoRA...")
!wget -q --show-progress -O /content/ComfyUI/models/loras/epiCRealismHelper.safetensors \
  "https://civitai.com/api/download/models/134065?type=Model&format=SafeTensor"

print("✅ All models downloaded!")

# ------------------------------
# START COMFYUI SERVER IN BACKGROUND
# ------------------------------
%cd /content/ComfyUI

import subprocess, threading, time, requests

def start_comfy():
    process = subprocess.Popen(
        ["python", "main.py", "--listen", "0.0.0.0", "--port", "8188"],
        cwd="/content/ComfyUI",
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True
    )
    for line in process.stdout:
        print(line, end="")

thread = threading.Thread(target=start_comfy, daemon=True)
thread.start()

# ------------------------------
# WAIT UNTIL COMFYUI IS READY
# ------------------------------
print("\n⏳ Waiting for ComfyUI to start...")

ready = False
for i in range(60):  # wait up to 60 seconds
    try:
        r = requests.get("http://127.0.0.1:8188")
        if r.status_code == 200:
            ready = True
            print("✅ ComfyUI is ready!")
            break
    except:
        pass
    if i % 5 == 0:
        print(f"   Still waiting... ({i}s)")
    time.sleep(1)

if not ready:
    print("❌ ComfyUI failed to start in 60 seconds")
    print("Check the logs above for errors")
else:
    print("\n🌐 Starting Cloudflare tunnel...")

    # ------------------------------
    # INSTALL AND START CLOUDFLARED TUNNEL
    # ------------------------------
    !wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
    !dpkg -i cloudflared-linux-amd64.deb


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://cli.github.com/packages stable/main amd64 Packages [343 B]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,143 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,457 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu j

In [ ]:
# Replace with your actual token from Cloudflare Zero Trust dashboard
CLOUDFLARE_TOKEN = "eyJhIjoiZmY1NGQ0MmY2YTI4OTlhODkyMjhlZDQ3Zjc4NTk3N2MiLCJ0IjoiZTQ1NWY2ZjAtN2UxMi00NTcxLWI4ZjAtZWU1OThlNTc2OTg2IiwicyI6Ik5XRTFOekF4TVdZdFpUbGlaaTAwT0RVNExUZzFNbVF0TlRReE1tWmxOelkzTm1SbSJ9"  # <-- PASTE YOUR TOKEN HERE

if CLOUDFLARE_TOKEN and CLOUDFLARE_TOKEN.strip():
    print("🔗 Starting tunnel with your token...")

    # Start cloudflared tunnel in background
    import subprocess
    tunnel_process = subprocess.Popen(
        ["cloudflared", "tunnel", "run", "--protocol", "http2", "--url", "http://127.0.0.1:8188", "--token", CLOUDFLARE_TOKEN],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True
    )

    print("⏳ Waiting for tunnel to connect...")
    import time
    time.sleep(5)  # Give tunnel time to establish

    print("✅ Cloudflare tunnel started!")
    print("🌐 Access ComfyUI through your Cloudflare tunnel URL")
    print("   (Check your Cloudflare Zero Trust dashboard for the URL)")

    # Keep showing tunnel logs
    import threading
    def show_tunnel_logs():
        for line in tunnel_process.stdout:
            print(f"[Tunnel] {line}", end="")

    log_thread = threading.Thread(target=show_tunnel_logs, daemon=True)
    log_thread.start()

else:
    # Option 2: Quick tunnel (no token needed, but temporary URL)
    print("⚠️ No token provided. Starting quick tunnel (temporary URL)...")
    import subprocess
    import re

    tunnel_process = subprocess.Popen(
        ["cloudflared", "tunnel", "--url", "http://127.0.0.1:8188"],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True
    )

    print("⏳ Waiting for tunnel URL...")
    for line in tunnel_process.stdout:
        print(line, end="")
        # Look for the tunnel URL
        if "trycloudflare.com" in line:
            url_match = re.search(r'https://[a-z0-9-]+\.trycloudflare\.com', line)
            if url_match:
                tunnel_url = url_match.group(0)
                print(f"\n✅ ComfyUI is now accessible at: {tunnel_url}")
                print(f"   Open this URL in your browser to use ComfyUI")
                break

print("\n" + "="*60)
print("🎉 Setup complete!")
print("="*60)
print("\nYour workflow requires these images:")
print("  1. character_reference.jpg - Clear face photo")
print("  2. outfit_reference.jpg - Clothing reference")
print("  3. pose_reference.jpg - Pose image (optional)")
print("  4. location_reference.jpg - Background (optional)")
print("  5. product_reference.jpg - Product image (optional)")
print("\nUpload these images in ComfyUI and load the workflow JSON!")

🔗 Starting tunnel with your token...
⏳ Waiting for tunnel to connect...
FETCH ComfyRegistry Data: 10/106
FETCH ComfyRegistry Data: 15/106
✅ Cloudflare tunnel started!
🌐 Access ComfyUI through your Cloudflare tunnel URL
   (Check your Cloudflare Zero Trust dashboard for the URL)

🎉 Setup complete!

Your workflow requires these images:
  1. character_reference.jpg - Clear face photo
  2. outfit_reference.jpg - Clothing reference
  3. pose_reference.jpg - Pose image (optional)
  4. location_reference.jpg - Background (optional)
  5. product_reference.jpg - Product image (optional)

Upload these images in ComfyUI and load the workflow JSON!
[Tunnel] 2025-11-16T07:08:07Z INF Starting tunnel tunnelID=e455f6f0-7e12-4571-b8f0-ee598e576986
[Tunnel] 2025-11-16T07:08:07Z INF Version 2025.11.1 (Checksum 83ea55259e419549817460d0c097f23ad1327364d0a63fab2c5463b9283251cb)
[Tunnel] 2025-11-16T07:08:07Z INF GOOS: linux, GOVersion: go1.24.9, GoArch: amd64
[Tunnel] 2025-11-16T07:08:07Z INF Settings: map[p